In [4]:
import pandas as pd
import json

# Load the files
with open('products.json', 'r', encoding='utf-8') as f:
    products = json.load(f)

with open('qnas.json', 'r', encoding='utf-8') as f:
    qnas = json.load(f)

with open('reviews.json', 'r', encoding='utf-8') as f:
    reviews = json.load(f)

# Convert JSON data to DataFrames
products_df = pd.DataFrame.from_dict(products, orient='index')
qnas_df = pd.DataFrame.from_dict(qnas, orient='index')
reviews_df = pd.DataFrame.from_dict(reviews, orient='index')

# Clean the keys by removing query parameters for consistency
qnas_df['product_id'] = qnas_df['product_id'].str.split('?').str[0]
reviews_df['product_id'] = reviews_df['product_id'].str.split('?').str[0]

# Perform the first merge
intermediate_merge = pd.merge(products_df, qnas_df, how='left', left_on='id', right_on='product_id')

# Perform the second merge
final_merged_df = pd.merge(intermediate_merge, reviews_df, how='left', left_on='id_x', right_on='product_id')

# Save the final merged dataset to a CSV file
final_merged_df.to_csv('merged_data.csv', index=False)

# Print the first few rows of the dataset to inspect
print(final_merged_df.head())


                             category_id  \
0  https://cellphones.com.vn/laptop.html   
1  https://cellphones.com.vn/laptop.html   
2  https://cellphones.com.vn/laptop.html   
3  https://cellphones.com.vn/laptop.html   
4  https://cellphones.com.vn/laptop.html   

                                         description  total_review  \
0  Nguyên hộp, đầy đủ phụ kiện từ nhà sản suất\nB...             0   
1  Nguyên hộp, đầy đủ phụ kiện từ nhà sản suất\nB...             0   
2  Nguyên hộp, đầy đủ phụ kiện từ nhà sản suất\nB...             0   
3  Nguyên hộp, đầy đủ phụ kiện từ nhà sản suất\nB...             0   
4  Nguyên hộp, đầy đủ phụ kiện từ nhà sản suất\nB...             0   

                                                id_x  \
0  cellphones_com_vn_laptop-acer-aspire-3-spin-14...   
1  cellphones_com_vn_laptop-acer-aspire-3-spin-14...   
2  cellphones_com_vn_laptop-acer-aspire-3-spin-14...   
3  cellphones_com_vn_laptop-acer-aspire-3-spin-14...   
4  cellphones_com_vn_laptop-acer-a

In [5]:
import pandas as pd
import json

# Load the files
with open('products.json', 'r', encoding='utf-8') as f:
    products = json.load(f)

with open('qnas.json', 'r', encoding='utf-8') as f:
    qnas = json.load(f)

with open('reviews.json', 'r', encoding='utf-8') as f:
    reviews = json.load(f)

# Convert JSON data to DataFrames
products_df = pd.DataFrame.from_dict(products, orient='index')
qnas_df = pd.DataFrame.from_dict(qnas, orient='index')
reviews_df = pd.DataFrame.from_dict(reviews, orient='index')

# Clean the keys by removing query parameters for consistency
qnas_df['product_id'] = qnas_df['product_id'].str.split('?').str[0]
reviews_df['product_id'] = reviews_df['product_id'].str.split('?').str[0]

# Perform the first merge
intermediate_merge = pd.merge(products_df, qnas_df, how='left', left_on='id', right_on='product_id')

# Perform the second merge
final_merged_df = pd.merge(intermediate_merge, reviews_df, how='left', left_on='id_x', right_on='product_id')

# Save the final merged dataset to a JSON file
final_merged_dict = final_merged_df.to_dict(orient='records')  # Convert to list of dictionaries
with open('merged_data.json', 'w', encoding='utf-8') as f:
    json.dump(final_merged_dict, f, ensure_ascii=False, indent=4)

# Print confirmation
print("Merged data saved to 'merged_data.json'")


Merged data saved to 'merged_data.json'


In [6]:
import pandas as pd
import json

# Load the dataset
data = pd.read_csv("merged_data.csv")

# Sanitize and parse the 'spec' column
def sanitize_and_parse_spec(spec_value):
    if isinstance(spec_value, dict):  # Already a dictionary
        return spec_value
    elif isinstance(spec_value, str):  # If it's a string
        try:
            sanitized_string = spec_value.replace("'", '"').replace("\\", "\\\\")
            return json.loads(sanitized_string)
        except Exception as e:
            return {}  # Return empty dictionary if parsing fails
    else:
        return {}  # Default to empty dictionary for other data types

# Ensure the 'spec' column exists and process it
if 'spec' in data.columns:
    data['spec_cleaned'] = data['spec'].apply(sanitize_and_parse_spec)
else:
    print("Column 'spec' not found in the dataset.")

# Extract specific fields from the sanitized 'spec_cleaned' column
if 'spec_cleaned' in data.columns:
    data['RAM'] = data['spec_cleaned'].apply(lambda x: x.get("Bộ nhớ RAM, Ổ cứng", {}).get("Dung lượng RAM", None))
    data['Storage'] = data['spec_cleaned'].apply(lambda x: x.get("Bộ nhớ RAM, Ổ cứng", {}).get("Ổ cứng", None))
    data['Processor'] = data['spec_cleaned'].apply(lambda x: x.get("Bộ xử lý & Đồ họa", {}).get("Loại CPU", None))
else:
    print("Column 'spec_cleaned' was not created.")

# Save the cleaned dataset to a new CSV file
cleaned_csv_path = "cleaned_data_fixed.csv"
data.to_csv(cleaned_csv_path, index=False)

# Display the first few rows of the cleaned dataset
print(data.head())


                             category_id  \
0  https://cellphones.com.vn/laptop.html   
1  https://cellphones.com.vn/laptop.html   
2  https://cellphones.com.vn/laptop.html   
3  https://cellphones.com.vn/laptop.html   
4  https://cellphones.com.vn/laptop.html   

                                         description  total_review  \
0  Nguyên hộp, đầy đủ phụ kiện từ nhà sản suất\nB...             0   
1  Nguyên hộp, đầy đủ phụ kiện từ nhà sản suất\nB...             0   
2  Nguyên hộp, đầy đủ phụ kiện từ nhà sản suất\nB...             0   
3  Nguyên hộp, đầy đủ phụ kiện từ nhà sản suất\nB...             0   
4  Nguyên hộp, đầy đủ phụ kiện từ nhà sản suất\nB...             0   

                                                id_x  \
0  cellphones_com_vn_laptop-acer-aspire-3-spin-14...   
1  cellphones_com_vn_laptop-acer-aspire-3-spin-14...   
2  cellphones_com_vn_laptop-acer-aspire-3-spin-14...   
3  cellphones_com_vn_laptop-acer-aspire-3-spin-14...   
4  cellphones_com_vn_laptop-acer-a

In [7]:
# Extract the primary image from 'image_url' column
def extract_primary_image(image_column):
    try:
        if pd.notnull(image_column):
            images = json.loads(image_column.replace("'", '"'))
            return next(iter(images.values()), "No image available")
        else:
            return "No image available"
    except Exception:
        return "No image available"

# Add 'Primary_Image' to the dataset
data['Primary_Image'] = data['image_url'].apply(extract_primary_image)

# Redefine the relevant columns and clean the dataset
relevant_columns = [
    'name', 'description', 'price', 'special_price', 'RAM', 'Storage', 'Processor',
    'Primary_Image', 'question', 'answer'
]
cleaned_data = data[relevant_columns]

# Fill missing values with appropriate placeholders
cleaned_data['description'] = cleaned_data['description'].fillna("Description not available")
cleaned_data['price'] = cleaned_data['price'].fillna(0)  # Set missing prices to 0
cleaned_data['special_price'] = cleaned_data['special_price'].fillna(cleaned_data['price'])  # Default to price
cleaned_data['RAM'] = cleaned_data['RAM'].fillna("Not specified")
cleaned_data['Storage'] = cleaned_data['Storage'].fillna("Not specified")
cleaned_data['Processor'] = cleaned_data['Processor'].fillna("Not specified")
cleaned_data['Primary_Image'] = cleaned_data['Primary_Image'].fillna("No image available")
cleaned_data['question'] = cleaned_data['question'].fillna("No question provided")
cleaned_data['answer'] = cleaned_data['answer'].fillna("No answer available")

# Standardize text fields
cleaned_data['description'] = cleaned_data['description'].str.strip()
cleaned_data['question'] = cleaned_data['question'].str.strip()
cleaned_data['answer'] = cleaned_data['answer'].str.strip()

# Add a calculated field for discount percentage
cleaned_data['discount_percentage'] = (
    ((cleaned_data['price'] - cleaned_data['special_price']) / cleaned_data['price']) * 100
).fillna(0).round(2)

# Save the cleaned dataset to a new CSV file
cleaned_data_path = 'usage_data.csv'
cleaned_data.to_csv(cleaned_data_path, index=False)

# Display the first few rows of the cleaned dataset
print(data.head())


                             category_id  \
0  https://cellphones.com.vn/laptop.html   
1  https://cellphones.com.vn/laptop.html   
2  https://cellphones.com.vn/laptop.html   
3  https://cellphones.com.vn/laptop.html   
4  https://cellphones.com.vn/laptop.html   

                                         description  total_review  \
0  Nguyên hộp, đầy đủ phụ kiện từ nhà sản suất\nB...             0   
1  Nguyên hộp, đầy đủ phụ kiện từ nhà sản suất\nB...             0   
2  Nguyên hộp, đầy đủ phụ kiện từ nhà sản suất\nB...             0   
3  Nguyên hộp, đầy đủ phụ kiện từ nhà sản suất\nB...             0   
4  Nguyên hộp, đầy đủ phụ kiện từ nhà sản suất\nB...             0   

                                                id_x  \
0  cellphones_com_vn_laptop-acer-aspire-3-spin-14...   
1  cellphones_com_vn_laptop-acer-aspire-3-spin-14...   
2  cellphones_com_vn_laptop-acer-aspire-3-spin-14...   
3  cellphones_com_vn_laptop-acer-aspire-3-spin-14...   
4  cellphones_com_vn_laptop-acer-a

C:\Users\lethi\AppData\Local\Temp\ipykernel_6596\647672142.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['description'] = cleaned_data['description'].fillna("Description not available")
C:\Users\lethi\AppData\Local\Temp\ipykernel_6596\647672142.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['price'] = cleaned_data['price'].fillna(0)  # Set missing prices to 0
C:\Users\lethi\AppData\Local\Temp\ipykernel_6596\647672142.py:25: SettingWithCopyWarning: 
A value is trying to

In [8]:
import pandas as pd
file_path = "usage_data.csv"
data = pd.read_csv(file_path)
json_output_path = 'usage_data.json'
data.to_json(json_output_path, orient='records', lines=True)

json_output_path

'usage_data.json'